In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import glob

In [2]:
basepath = "C:/Users/Judit/Desktop/uab/quart/tfg/dades_tfg/data"
demographics = pd.read_csv(os.path.join(basepath, "demographics.csv"))
nodes = pd.read_csv(os.path.join(basepath, "nodes.csv"))
basepath_FA = os.path.join(basepath, "FA")
basepath_GM = os.path.join(basepath, "GM")
basepath_RS = os.path.join(basepath, "RS")

In [3]:
filenames = ["{:04d}.csv".format(x) for x in demographics["id"]]
filenames

['0000.csv',
 '0001.csv',
 '0002.csv',
 '0003.csv',
 '0004.csv',
 '0005.csv',
 '0006.csv',
 '0007.csv',
 '0008.csv',
 '0009.csv',
 '0010.csv',
 '0011.csv',
 '0012.csv',
 '0013.csv',
 '0014.csv',
 '0015.csv',
 '0016.csv',
 '0017.csv',
 '0018.csv',
 '0019.csv',
 '0020.csv',
 '0021.csv',
 '0022.csv',
 '0023.csv',
 '0024.csv',
 '0025.csv',
 '0026.csv',
 '0027.csv',
 '0028.csv',
 '0029.csv',
 '0030.csv',
 '0031.csv',
 '0032.csv',
 '0033.csv',
 '0034.csv',
 '0035.csv',
 '0036.csv',
 '0037.csv',
 '0038.csv',
 '0039.csv',
 '0040.csv',
 '0041.csv',
 '0042.csv',
 '0043.csv',
 '0044.csv',
 '0045.csv',
 '0046.csv',
 '0047.csv',
 '0048.csv',
 '0049.csv',
 '0050.csv',
 '0051.csv',
 '0052.csv',
 '0053.csv',
 '0054.csv',
 '0055.csv',
 '0056.csv',
 '0057.csv',
 '0058.csv',
 '0059.csv',
 '0060.csv',
 '0061.csv',
 '0062.csv',
 '0063.csv',
 '0064.csv',
 '0065.csv',
 '0066.csv',
 '0067.csv',
 '0068.csv',
 '0069.csv',
 '0070.csv',
 '0071.csv',
 '0072.csv',
 '0073.csv',
 '0074.csv',
 '0075.csv',
 '0076.csv',

In [4]:
data_FA = np.zeros(shape=(len(filenames), 76, 76))
data_GM = np.zeros(shape=(len(filenames), 76, 76))
data_RS = np.zeros(shape=(len(filenames), 76, 76))

In [5]:
for i, filename in enumerate(filenames):
    df_FA = pd.read_csv(os.path.join(basepath_FA, filename), header=None)
    data_FA[i,:,:] = df_FA.values
    
    df_GM = pd.read_csv(os.path.join(basepath_GM, filename), header=None)
    data_GM[i,:,:] = df_GM.values
    
    df_RS = pd.read_csv(os.path.join(basepath_RS, filename), header=None)
    data_RS[i,:,:] = df_RS.values

In [6]:
data_FA

array([[[0.        , 0.40522193, 0.        , ..., 0.        ,
         0.        , 0.38546734],
        [0.40522193, 0.        , 0.        , ..., 0.44987853,
         0.        , 0.46430682],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.44987853, 0.        , ..., 0.        ,
         0.36835478, 0.35931725],
        [0.        , 0.        , 0.        , ..., 0.36835478,
         0.        , 0.35820915],
        [0.38546734, 0.46430682, 0.        , ..., 0.35931725,
         0.35820915, 0.        ]],

       [[0.        , 0.33989526, 0.        , ..., 0.        ,
         0.        , 0.37613276],
        [0.33989526, 0.        , 0.        , ..., 0.32421487,
         0.        , 0.37474739],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.32421487, 0.        , ..., 0.        ,
         0.34255555, 0.33477336],
        [0. 

In [7]:
data_GM

array([[[0.        , 0.01092925, 0.00976027, ..., 0.02400776,
         0.00720083, 0.02324084],
        [0.01092925, 0.        , 0.02157368, ..., 0.0530924 ,
         0.01532649, 0.05183536],
        [0.00976027, 0.02157368, 0.        , ..., 0.04686056,
         0.01387286, 0.04438257],
        ...,
        [0.02400776, 0.0530924 , 0.04686056, ..., 0.        ,
         0.03466667, 0.11354242],
        [0.00720083, 0.01532649, 0.01387286, ..., 0.03466667,
         0.        , 0.03288942],
        [0.02324084, 0.05183536, 0.04438257, ..., 0.11354242,
         0.03288942, 0.        ]],

       [[0.        , 0.01181403, 0.00701129, ..., 0.0354831 ,
         0.00705333, 0.02451802],
        [0.01181403, 0.        , 0.01516767, ..., 0.07860736,
         0.01521611, 0.05506321],
        [0.00701129, 0.01516767, 0.        , ..., 0.04586335,
         0.00918694, 0.03160368],
        ...,
        [0.0354831 , 0.07860736, 0.04586335, ..., 0.        ,
         0.04680758, 0.160669  ],
        [0.0

In [8]:
data_RS

array([[[0.        , 0.32148423, 0.22454527, ..., 0.18503865,
         0.43315624, 0.33278617],
        [0.32148423, 0.        , 0.71249905, ..., 0.49187673,
         0.65304696, 0.44755928],
        [0.22454527, 0.71249905, 0.        , ..., 0.39944373,
         0.63349742, 0.48849575],
        ...,
        [0.18503865, 0.49187673, 0.39944373, ..., 0.        ,
         0.27314513, 0.55031085],
        [0.43315624, 0.65304696, 0.63349742, ..., 0.27314513,
         0.        , 0.60976677],
        [0.33278617, 0.44755928, 0.48849575, ..., 0.55031085,
         0.60976677, 0.        ]],

       [[0.        , 0.02130933, 0.09358542, ..., 0.29462944,
         0.07273126, 0.28249394],
        [0.02130933, 0.        , 0.17566425, ..., 0.22308508,
         0.33301901, 0.0437634 ],
        [0.09358542, 0.17566425, 0.        , ..., 0.00373577,
         0.04172906, 0.16673171],
        ...,
        [0.29462944, 0.22308508, 0.00373577, ..., 0.        ,
         0.37736777, 0.20350557],
        [0.0